# Implementation of MST 2018 ECMA
Original paper:
- Mogstad, Magne, Andres Santos, and Alexander Torgovitsky. "Using instrumental variables for inference about policy relevant treatment parameters." Econometrica 86.5 (2018): 1589-1619.


For an implementation in `R` see
- Shea, Joshua, and Alexander Torgovitsky. "ivmte: An R Package for Extrapolating Instrumental Variable Estimates Away From Compliers." Observational Studies 9.2 (2023): 1-42.

For an implementation in `julia` see

- https://github.com/a-torgovitsky/MarginalTreatmentEffects.jl

## Setup

- target estimand
- set of identified estimands
- parameter space
  - use finite-dimensional approximation
  - for some spaces have exact results using splines

### IV-like specifications and $M_S$
$$ \Gamma_s(m) := E\left[\int_0^1 m_0(u,X)\omega_{0s}(u,Z)du\right] + E\left[\int_0^1 m_1(u,X)\omega_{1s}(u,Z)du\right] $$
where the weights are given by $\omega_{0s} := s(0,z)I[u>p(z)]$ and $\omega_{1s} := s(1,z)I[u\leq p(z)]$. The functions $s(d,z)$ are implied by the given IV-like estimand.

Under the assumptions of the model, the MTR functions $m:=(m_0, m_1)$ must satisfy $\Gamma_s(m) =\beta_s$ for each IV-like specification $s\in \mathcal{S}$.

Thus, $m\in M_{\mathcal{S}} := \{m\in M: \Gamma_s(m) = \beta_s \text{ for all } s\in\mathcal{S}\}$.

### Target Estimand
Equivalently, we can define the linear map $\Gamma^*:M\to R$ by
$$ \Gamma^*(m) := E\left[\int_0^1 m_0(u,X)\omega_{0s}^*(u,Z)d\mu^*(u)\right] + E\left[\int_0^1 m_1(u,X)\omega_{1s}^*(u,Z)d\mu^*(u)\right] $$

Similarly, under the model assumptions we must have that the target parameter belongs to $B_S^* := \{b\in \mathbb{R}: b = \Gamma^*(m) \text{ for some } m\in M_S\}$.

### Optimization Problem
Proposition 2 in the paper says that if $M$ is convex, either $B_S^*$ is empty or its closure is (in $\mathbb{R}$) is equal to the interval $[\underline{\beta}^*, \overline{\beta}^*]$ where
$\underline{\beta}^* := \inf_{m\in M_S}\Gamma^*(m)$ and $\overline{\beta}^* := \sup_{m\in M_S}\Gamma^*(m)$.

## Implementation
The paper discusses implementation in Section 3. The idea is to replace the possible infinite-dimensional parameter space of functions given by $M$ with a finite-dimensional $M_{fd}\subseteq M$. In particular, they propose a finite linear basis
$$ M_{fd} := \left\{ (m_0, m_1) \in M: m_d(u,x) = \sum_{k=1}^{K_d}\theta_{dk}b_{dk}(u,x) \text{ for some } \{\theta_{dk}\}_{k=1}^{K_d}, d=0,1 \right\}.$$

Thus, we moved to a finite dimensional space of functions that is characterized by $\theta := (\theta_0', \theta_1')$ and known basis functions $\{b_{dk}\}_{k=1}^{K_d}$.

Thus, $M_{fd}$ implies a parameter space made of elements $\theta \in R^{K_0}\times R^{K_1}$
$$ \Theta := \left\{(\theta_0, \theta_1)\in \mathbb{R}^{K_0} \times \mathbb{R}^{K_1}: \left(\sum_{k=1}^{K_0}\theta_{0k}b_{0k}, \sum_{k=1}^{K_1}\theta_{1k}b_{1k},\right) \in M \right\} $$

### Linear Program
We can then use the linearity of the mappings $\Gamma^*$ and $\Gamma_s$ to write the optimization problem as
$$ \overline{\beta}^*_{fd} := \sup_{(\theta_0, \theta_1)\in \Theta} \sum_{k=1}^{K_0} \theta_{0k}\Gamma_0^*(b_{0k}) + \sum_{k=1}^{K_1} \theta_{1k} \Gamma_1^*(b_{1k}).$$

subject to the constraints that $\sum_{k=1}^{K_0}\theta_{0k}\Gamma_{0s}(b_{0k}) + \sum_{k=1}^{K_1}\theta_{1k}\Gamma_{1s}(b_{1k}) = \beta_s$ for all $s\in S$. This is the linear program we want to solve.

In [3]:
from amplpy import AMPL
import pandas as pd